# Notebook para responder a parte 3 do PP1 de RNA

### 3 Tipos de Tarefas (Dayvson)
Recapitulem que dados fornecem experiência sobre um problema. No caso em questão, sugira:
- [x] Uma tarefa de classificação mediante Aprendizado Supervisionado que poderia ser feita com
esta base de dados. Qual seria o atributo-alvo? Quais métricas de desempenho poderiam ser
aplicadas? Que tipo de validação seria apropriado?
- [ ] Uma tarefa de regressão mediante Aprendizado Supervisionado que poderia ser feita com
esta base de dados. Qual seria o atributo-alvo? Quais atributos preditores a equipe considera
relevantes para o cenário?
- [ ] Bônus: Qual tarefa de Aprendizado Não-Supervisionado poderia ser concebida neste contexto?

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import re


In [8]:
file = '/home/dayvson/disc/rna/Analise-COVID19-PP1-RNA2020.1/dataset/dataset_limpo_covid19_manaus.csv'
pd.read_csv(file)

,idade,sexo,bairro,conclusao,dt_notificacao,tipo_teste
0,25,F,ADRIANOPOLIS,Recuperado,20/03/2020,RT-PCR
1,41,F,BETANIA,Recuperado,27/03/2020,RT-PCR
2,37,M,CIDADE NOVA,Recuperado,27/04/2020,TESTE RÁPIDO - ANTICORPO
3,44,M,SAO JORGE,Recuperado,27/04/2020,TESTE RÁPIDO - ANTICORPO
4,27,F,COLONIA TERRA NOVA,Recuperado,24/04/2020,TESTE RÁPIDO - ANTICORPO
...,...,...,...,...,...,...
6355,14,F,DA PAZ,Recuperado,25/06/2020,TESTE RÁPIDO - ANTÍGENO
6356,5,F,REDENCAO,Recuperado,18/06/2020,TESTE RÁPIDO - ANTÍGENO
6357,70,F,DOM PEDRO I,Recuperado,02/06/2020,TESTE RÁPIDO - ANTÍGENO
6358,68,M,JORGE TEIXEIRA,Recuperado,01/06/2020,RT-PCR
